Jupyter notebook contains analysis for investigating what two galaxies, the MW and GES look like during their merger. This includes plotting images of histograms and creating gifs, and finding and tracking the centres of the galaxies in different ways during the merger.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Auriga_Analysis as A
import imageio

plt.rcParams['text.usetex'] = True

In [ ]:
MW_ids = np.load('Data/MW_ids_stellarhalo_1194.npy')
GES_ids = np.load('Data/GES_ids_stellarhalo_1194.npy')
GES_position =  np.load('Data/GES_positions.npy')
GES_snaps = np.load('Data/GES_snaps.npy')
subfind = np.load('/Data/SubfindMT.npy')

In [ ]:
filenames = []
count = 0

b = 0.03
f = 10**3

snap = 1700

positions,masses,ids = A.SnapshotData.load_snapshot(snap,4,['Coordinates', 'Masses', 'ParticleIDs']).get_results()

MWcentre =  A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

if snap in GES_snaps:
    indice = list(GES_snaps).index(snap)
    GEScentre = GES_position[indice]
    GEScentred = A.util.CentreOnHalo(GEScentre, MWcentre)

else:
    GEScentred = A.util.CentreOnHalo(MWcentre, MWcentre)

time = A.util.time(snap)
print(time)

idx = np.where(np.isin(ids, MW_ids))[0]
coords = positions[idx,:]
coords = A.util.CentreOnHalo(coords, MWcentre)
    
n, xedges, yedges = np.histogram2d(coords[:,0], coords[:,1], bins=(500,500),range=[[-b, b],[-b, b]] )
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
xc, yc = np.meshgrid(xbin, ybin)
        
fig, ax1 = plt.subplots(figsize=(4,4))
# fig.dpi = 300

ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma', alpha = 0.2)

idx = np.where(np.isin(ids, GES_ids))[0]
coords = positions[idx,:]
coords = A.util.CentreOnHalo(coords, MWcentre)

n, xedges, yedges = np.histogram2d(coords[:,0], coords[:,1], bins=(500,500),range=[[-b, b],[-b, b]] )
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
xc, yc = np.meshgrid(xbin, ybin)
    
ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')
ax1.scatter(0,0, label = 'MW centre',s = 18, color = 'black')
ax1.scatter(GEScentred[0]*f , GEScentred[1]*f, s= 6, color = 'springgreen', label = 'GES centre')

ax1.set_xlim(-b*f,b*f)
ax1.set_ylim(-b*f,b*f)

# plt.title(r'$\mathrm{5.68 \, Gyr}$', fontsize =20)
plt.title('Time = {} Gyr'.format(np.round(time,2)))

ax1.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False, 
                labelbottom=False, labelleft=False)

In [ ]:
def particles_within_sphere(centre, radius, pos, masses):

    distances = np.linalg.norm(pos - centre, axis=1)  
    mask = distances <= radius 
    
    filtered_pos = pos[mask]
    filtered_mass = masses[mask]
    return filtered_pos, filtered_mass

def barycentre(positions, masses):

    centres = []
    
    for i in range(3):    #used to be for i in range(3) in case shrinking spheres in 3D stops working
        cm = np.sum(positions[:,i]*masses)/sum(masses)
        centres.append(cm)
    return np.array(centres)

def density(masses, radius):
    dense = np.sum(masses)/((4/3)*np.pi*radius**3)
    return dense

In [ ]:
def shrinking_spheres(pos, masses,radius, r,no_part, centre):

    num_particles = len(pos)
    no_iterations = 0
    
    while num_particles > no_part:

        distances = np.linalg.norm(pos - centre, axis=1)  
        mask = distances <= radius 
        
        pos = pos[mask]
        masses = masses[mask]
    
        num_particles = len(pos)
        centre = np.sum(pos * masses[:, np.newaxis], axis=0) / np.sum(masses)
        
        radius = radius*r
    
        no_iterations +=1
        
    dense = density(masses,radius)
    
    return(pos,masses, no_iterations, radius,centre,dense)

In [ ]:
GEScentres = []
centres = []
iterations = []
densities = []
radii = []
times = []

for i in range(1196, 2999,4):
    snap = i
    if snap%100 == 0:
        print(snap)
    times.append(A.util.time(snap))
    positions,masses,ids = A.SnapshotData.load_snapshot(snap,4,['Coordinates', 'Masses', 'ParticleIDs']).get_results()
    
    MWcentre =  A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()
    radius =  A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()

    if snap in GES_snaps:
        indice = list(GES_snaps).index(snap)
        GEScentre = GES_position[indice]
        GEScentred = A.util.CentreOnHalo(GEScentre, MWcentre)

    else:
        GEScentred = A.util.CentreOnHalo(MWcentre, MWcentre)

    GEScentres.append(GEScentred)
    
    idx = np.where(np.isin(ids, GES_ids))[0]
    coords = positions[idx,:]
    masses = masses[idx,]
    coords = A.util.CentreOnHalo(coords, MWcentre)

    pos,masses, no_iterations, radius,centre,dense = shrinking_spheres(coords,masses,0.05, 0.85, 1000, GEScentred)
    iterations.append(no_iterations)
    radii.append(radius)
    centres.append(centre)
    densities.append(dense)

In [ ]:
frac_dense = (np.array(densities)/density_of_sphere)*100

plt.figure(figsize = (5,5))

plt.plot(times, np.array(radii)*10**3)

#plt.plot(times, iterations)

plt.xlabel('Time / Gyr', fontsize = 12)
plt.ylabel('Sphere radius / kpc', fontsize = 12)

# plt.xlim(5.2, 6.8)
plt.ylim(0,5.1)

In [ ]:
plt.figure(figsize = (5,5))
plt.plot(times, densities)
plt.xlim(5.5,13.8)
plt.xlabel('Time / Gyr')
plt.ylabel(r'Density of sphere  / $M_\odot / \mathrm{Mpc}^{-3}$')

In [ ]:
centres = np.array(centres)
GES = np.array(GEScentres)

x = 1

plt.plot(times,centres[:,x]*10**3)
plt.plot(times, GES[:,x]*10**3)

In [ ]:
plt.figure(figsize = (8,5))

diff = abs(GES-centres)
difference = np.sqrt(diff[:,0]**2+diff[:,1]**2+diff[:,2]**2)
plt.plot(times, np.array(difference)*10**3)

plt.xlabel('Time / Gyr', fontsize = 12)
plt.ylabel('Distance to SUBFIND GES centre / kpc', fontsize = 12)#

In [ ]:
a_s = [A.util.scale_fac(snap) for snap in range(1196, 2999,4)]

In [ ]:
plt.figure(figsize = (8,5))

distance = np.sqrt(centres[:,0]**2+centres[:,1]**2+centres[:,2]**2)
distance = distance/0.6777

GES_distance = np.sqrt(GES[:,0]**2+GES[:,1]**2+GES[:,2]**2)
GES_distance = GES_distance/0.6777

# make physical instead of comoving
GES_physical_distance = GES_distance*np.array(a_s)
physical_distance = distance*np.array(a_s)

fig,ax = plt.subplots(figsize = (10,2.5))
# fig.dpi = 300

t_p = 4.945
t_m = 5.345

ax.plot(times, distance*10**3, label = r'$\mathrm{Shrinking \,spheres}$')
ax.plot(times, GES_distance*10**3, linestyle = '--', label = r'$\mathrm{SUBFIND}$')

ax.set_xlabel(r'$\mathrm{Time\, / \,Gyr}$', fontsize = 15)
ax.set_ylabel(r'$\mathrm{3D\, offset\, / \,kpc}$', fontsize = 15)

ax.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)

plt.axvline(x = t_p, color = 'silver', linestyle = '--')
plt.axvline(x = t_m, color = 'silver', linestyle = '--')

ax.legend(fontsize = 11, frameon = False)
plt.annotate(r'$t_p$',fontsize = 14, xy = (t_p, 0), xytext = (t_p, -4), arrowprops=dict(arrowstyle='-', color='black'), ha = 'center')
plt.annotate(r'$t_m$',fontsize = 14, xy = (t_m, 0), xytext = (t_m, -4), arrowprops=dict(arrowstyle='-', color='black'), ha = 'center')

ax.set_ylim(0,40)
ax.set_xlim(4.8,14)

plt.show()

In [ ]:
filenames = []
count = 0

for i in range(len(centres)):

    fig = plt.figure(figsize = (4,4))
    ax = fig.add_subplot(projection='3d')
        
    p = ax.scatter(centres[i][0]*f, centres[i][1]*f,centres[i][2]*f, s=10, color = 'black')
    p = ax.scatter(GES[i][0]*f, GES[i][1]*f,GES[i][2]*f, s=10, color = 'red')
    
    ax.set_xlabel('kpc')
    ax.set_ylabel('kpc')
    ax.set_zlabel('kpc')
    
    b = 20
    
    ax.set_xlim(-b,b)
    ax.set_ylim(-b, b)
    ax.set_zlim(-b, b)

    plt.title('Time = {} Gyr'.format(np.round(times[i], 2)))

    count +=1
    name = 'Images_for_gif/{}.png'.format(i)
    plt.savefig(name)
    filenames.append(name)

    plt.close()

In [ ]:
with imageio.get_writer('GES.gif', mode='I', fps = 3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
matrices = np.load('Data/Variable_rotation_matrices.npy')
rotmat_snaps = list(np.load('Data/Rotmat_snapshots.npy'))

In [ ]:
def snapshot_data(snap):

    time = A.util.time(snap)

    results = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses", "ParticleIDs"])
    coordinates, magnitudes, velocities, masses, ids = results.get_results()
    radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
    centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

    if snap in rotmat_snaps:
        centred = A.util.CentreOnHalo(coordinates, centre)
        velocities, bulk_velocity = A.util.remove_bulk_velocity(centred, velocities, masses, idx = None, radialcut = 0.5*radius)
        rotmat = matrices[rotmat_snaps.index(snap)]
        new_coords = np.dot(centred, rotmat)
        vels = np.dot(velocities, rotmat)
    
    else:
        new_coords, new_vels, coords, vels, masses, other,ids = A.util.align_withids(centre, coordinates, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], ids = [ids],to_print = False)
        magnitudes = other[0]

        ids = ids[0]

    return new_coords, vels, magnitudes, masses,ids, time

In [ ]:
coords, vels, mags,masses,ids, time = snapshot_data(1300)
n,xc, yc = A.Isophotes.make_SB_map(coords, 0.035, 128, mags, 3, 1.5, 10**3, time, show_plot = True)

In [ ]:
coords, vels, mags,masses, ids,time = snapshot_data(1300)
idx = np.concatenate([np.where(np.isin(ids, MW_ids))[0], np.where(np.isin(ids, GES_ids))[0]])
coords = coords[idx,:]
mags = mags[idx,]

n,xc, yc = A.Isophotes.make_SB_map(coords, 0.035, 128, mags, 3, 1.5, 10**3, time, show_plot = True)

In [ ]:
detail = 'SubhaloMassType'

tree1  = A.MergerTrees.load_snapshot(1000,1500,1,0, detail)   #for ascending end snap > beginning snap
results, snaps = tree1.walk_the_tree_ascending()
snapshots1 = [snap[0] for snap in snaps]
result1 = np.array([result[0] for result in results])

tree1  = A.MergerTrees.load_snapshot(1000,1500,0,0, detail)   #for ascending end snap > beginning snap
results, snaps = tree1.walk_the_tree_ascending()
snapshots2 = [snap[0] for snap in snaps]
result2 = np.array([result[0] for result in results])

detail = 'SubhaloPos'

tree3  = A.MergerTrees.load_snapshot(1000,1500,1,0, detail)   #for ascending end snap > beginning snap
results, snaps = tree3.walk_the_tree_ascending()
snapshots3 = [snap[0] for snap in snaps]
result3 = np.array([result[0] for result in results])

In [ ]:
MW_stellar = [result[4] for result in result2]
GES_stellarmass = [result[4] for result in result1]

MW_mass = [np.sum(result) for result in result2]
GES_mass = [np.sum(result) for result in result1]

GES_position = result3

MW_stellarmass = np.delete(MW_stellar, 223, 0)
MW_mass = np.delete(MW_mass, 223,0)

In [ ]:
snapshots = snapshots1
times = [A.util.time(snap) for snap in snapshots]

In [ ]:
GES_radius_stars = np.array([A.util.subhalo_radius(mass) for mass in GES_stellarmass])
MW_radius_stars = np.array([A.util.subhalo_radius(mass) for mass in MW_stellarmass])

GES_radius_all = np.array([A.util.subhalo_radius(mass) for mass in GES_mass])
MW_radius_all = np.array([A.util.subhalo_radius(mass) for mass in MW_mass])

In [ ]:
f = 10**3
plt.figure(figsize = (5,4))

plt.plot(times, MW_radius_stars*f)
plt.plot(times, GES_radius_stars*f)

plt.xlim(4.7, 4.9)
plt.ylim(17,17.5)

plt.xlabel('Time / Gyr')
plt.ylabel('Stellar subhalo radius / kpc')

plt.figure(figsize = (5,4))

plt.plot(times, MW_radius_all*f)
plt.plot(times, GES_radius_all*f)

plt.xlabel('Time / Gyr')
plt.ylabel('Subhalo radius / kpc')

In [ ]:
snap = 1194
indice = snapshots.index(snap)

subhalo = 'stellar'

f = 10**3

position =GES_position[snapshots.index(snap)]
centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()
radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
diff = position-centre
distance = np.sqrt(diff[0]**2+diff[1]**2+diff[2]**2)*f

print('Time = {} Gyr'.format(A.util.time(snap)))
print('Distance = {} / kpc'.format(distance))
print('Main halo r_200 = {} / kpc'.format(radius*f))

GES_centre = GES_position[indice]
MW_centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

if subhalo == 'stellar':
    MW_rad = MW_radius_stars[indice]
    GES_rad = GES_radius_stars[indice]
elif subhalo == 'all':
    MW_rad = MW_radius_all[indice]
    GES_rad = GES_radius_all[indice]

positions, masses, ids = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","Masses", "ParticleIDs"]).get_results()

def r(coords, centre):
    radius = np.sqrt((coords[:, 0] - centre[0])**2 + (coords[:, 1] - centre[1])**2 + (coords[:, 2] - centre[2])**2)
    return radius

radius = r(positions, GES_centre)
idx, = np.where(radius < GES_rad)

GES_ids = ids[idx,]
GES_positions = positions[idx,:]
GES_masses = masses[idx,]

radius = r(positions, MW_centre)
idx, = np.where(radius < MW_rad)

MW_ids = ids[idx,]
MW_positions = positions[idx,:]
MW_masses = masses[idx,]

print('Fraction of stellar mass contained within GES at this point = {}'. format(np.sum(GES_masses)/GES_stellarmass[snapshots.index(snap)]))
print('Fraction of stellar mass contained within MW at this point = {}'. format(np.sum(MW_masses)/MW_stellarmass[snapshots.index(snap)]))

In [ ]:
#centre on MW and align to its plane

matrices = np.load('Data/Variable_rotation_matrices.npy')
rotmat_snaps = list(np.load('Data/Rotmat_snapshots.npy'))

MW_positions = A.util.CentreOnHalo(MW_positions, MW_centre)
GES_positions = A.util.CentreOnHalo(GES_positions, MW_centre)

rotmat = matrices[rotmat_snaps.index(snap)]
MW_positions = np.dot(MW_positions, rotmat)
GES_positions = np.dot(GES_positions, rotmat)

In [ ]:
n, xedges, yedges = np.histogram2d(MW_positions[:,1], MW_positions[:,2], bins=(500,500))
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
xc, yc = np.meshgrid(xbin, ybin)
fig, ax1 = plt.subplots(figsize=(4, 4))

# fig.dpi = 300

ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')

n, xedges, yedges = np.histogram2d(GES_positions[:,1], GES_positions[:,2], bins=(500,500))
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
xc, yc = np.meshgrid(xbin, ybin)
        
ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')
ax1.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)


ax1.set_xlabel('kpc', fontsize = 12)
ax1.set_ylabel('kpc', fontsize = 12)

In [ ]:
#density profile of GES at this time
snap = 1194
indice = snapshots.index(snap)
f = 10**3

GES_positions = A.util.CentreOnHalo(GES_positions, GES_centre)
fig, ax1 = plt.subplots(figsize=(4, 4))
n, xedges, yedges = np.histogram2d(GES_positions[:,2], GES_positions[:,1], bins=(500,500))
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
xc, yc = np.meshgrid(xbin, ybin)

ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')

plt.xlabel('x / kpc')
plt.ylabel('y / kpc')

In [ ]:
plt.figure(figsize = (4,4))

r_bin_centers, radial_profile = A.Isophotes.radial_profile_func(n, xc, yc, num_bins=100)
plt.plot(r_bin_centers*f, radial_profile)

plt.xlabel('Radial distance / kpc')
plt.ylabel('Normalised stellar density')

In [ ]:
filenames = []
count = 0

b = 0.03
f = 10**3

for i in range(1270, 1273, 3):
    snap = i
    
    positions,masses,ids,vels = A.SnapshotData.load_snapshot(snap,4,['Coordinates', 'Masses', 'ParticleIDs', 'Velocities']).get_results()
    
    MWcentre =  A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()
    MWradius =  A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()

    if snap in GES_snaps:
        indice = list(GES_snaps).index(snap)
        GEScentre = GES_position[indice]
    else:
        GEScentre = MWcentre
        
    GEScentred = A.util.CentreOnHalo(GEScentre, MWcentre)

    time = A.util.time(snap)

    if snap in rotmat_snaps:
        rotmat = matrices[rotmat_snaps.index(snap)]
    else:
        coords = A.util.CentreOnHalo(positions, MWcentre)  # Centre on halo
        vels, bulk_velocity = A.util.remove_bulk_velocity(coords, vels, masses, radialcut=0.1*MWradius)
    
        radius = A.util.r(coords)
        idx, = np.where(radius < 0.1*MWradius)
    
        L = np.cross(coords[idx, :], vels[idx, :] * masses[idx, None])
        Ltot = L.sum(axis=0)
        Ldir = Ltot / np.sqrt((Ltot**2).sum())
    
        # Get principal axes and rotation matrix
        xdir, ydir, zdir = A.util.get_principal_axis(coords, masses, idx, L=Ldir)
    
        # Create rotation matrix
        matrix = np.array([xdir, ydir, zdir])
        rotmat = matrix.transpose()
    
    GEScentred = np.dot(GEScentred, rotmat)
    
    idx = np.where(np.isin(ids, MW_ids))[0]
    coords = positions[idx,:]
    coords = A.util.CentreOnHalo(coords, MWcentre)
    coords = np.dot(coords, rotmat)
    
    n, xedges, yedges = np.histogram2d(coords[:,1], coords[:,2], bins=(500,500),range=[[-b, b],[-b, b]] )
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
            
    fig, ax1 = plt.subplots(figsize=(5,5))
    
    # fig.dpi = 300
    
    ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma', alpha = 0.2)
    
    idx = np.where(np.isin(ids, GES_ids))[0]
    coords = positions[idx,:]
    coords = A.util.CentreOnHalo(coords, MWcentre)
    coords = np.dot(coords, rotmat)
    
    n, xedges, yedges = np.histogram2d(coords[:,1], coords[:,2], bins=(500,500),range=[[-b, b],[-b, b]] )
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
        
    ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')
    ax1.scatter(0,0, label = 'MW centre',s = 10, color = 'black')
    ax1.scatter(GEScentred[1]*f , GEScentred[2]*f, s= 6, color = 'blue', label = 'GES centre')
    
    ax1.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)
    
    ax1.set_xlim(-b*f,b*f)
    ax1.set_ylim(-b*f,b*f)
    
    ax1.set_xlabel('kpc', fontsize = 12)
    ax1.set_ylabel('kpc', fontsize = 12)
    
    ax1.legend(loc = 'upper right')

    if snap in [1299, 1300, 1301, 1302, 1303, 1304, 1305,1306, 1307]:
        plt.title('Time = {} Gyr \n MERGER'.format(np.round(time,2)), fontsize =14)
        
    else:     
        plt.title('Time = {} Gyr'.format(np.round(time,2)), fontsize =14)

    count+=1
    
    name = 'Images_for_gif/{}.png'.format(count)
    plt.savefig(name)
    # , dpi=300, bbox_inches='tight')    
    filenames.append(name)
    
    plt.close()

In [ ]:
with imageio.get_writer('merger.gif', mode='I', fps = 3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

## Using shrinking spheres

In [ ]:
def r(coords, centre):
    radius = np.sqrt((coords[:, 0] - centre[0])**2 + (coords[:, 1] - centre[1])**2 + (coords[:, 2] - centre[2])**2)
    return radius

def barycentre(positions, masses):

    centres = []
    
    for i in range(len(positions[0])):    #used to be for i in range(3) in case shrinking spheres in 3D stops working
        cm = np.sum(positions[:,i]*masses)/sum(masses)
        centres.append(cm)
    return np.array(centres)

def shrinking_spheres_stars(pos, masses, ids, radius, f,no_part, centre):

    num_particles = len(pos)

    no_iterations = 0
    
    while num_particles > no_part:

        distance = r(pos, centre)
        idx, = np.where(distance < radius)
        pos = pos[idx,]
        masses = masses[idx,]
        ids = ids[idx,]

        num_particles = len(pos)

        centre = barycentre(pos,masses)
        no_iterations+=1
        radius = radius*f
        
    return(pos,masses,ids, no_iterations, radius,centre)

In [ ]:
snap = 1194
ss_n = 5000

indice = snapshots.index(snap)

subhalo = 'stellar'

f = 10**3

position =GES_position[snapshots.index(snap)]
centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()
radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
diff = position-centre
distance = np.sqrt(diff[0]**2+diff[1]**2+diff[2]**2)*f

print('Time = {} Gyr'.format(A.util.time(snap)))
print('Distance = {} / kpc'.format(distance))
print('Main halo r_200 = {} / kpc'.format(radius*f))

GES_centre = GES_position[indice]
MW_centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

if subhalo == 'stellar':
    MW_rad = MW_radius_stars[indice]
    GES_rad = GES_radius_stars[indice]
elif subhalo == 'all':
    MW_rad = MW_radius_all[indice]
    GES_rad = GES_radius_all[indice]

positions, masses, ids = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","Masses", "ParticleIDs"]).get_results()

radius = r(positions, GES_centre)
idx, = np.where(radius < GES_rad)

GES_ids = ids[idx,]
GES_positions = positions[idx,:]
GES_masses = masses[idx,]

#now perform shrinking spheres to find densest n stars in GES

GES_positions1,GES_masses1,GES_ids1, no_iterations1, ss_radius1,centre1 = shrinking_spheres_stars(GES_positions, GES_masses, GES_ids, GES_rad, 0.75,5000, GES_centre)
print('Shrinking spheres radius of GES = {} kpc with n = {}'.format(ss_radius1*f, 5000))

GES_positions2,GES_masses2,GES_ids2, no_iterations2, ss_radius2,centre2 = shrinking_spheres_stars(GES_positions, GES_masses, GES_ids, GES_rad, 0.75,1000, GES_centre)
print('Shrinking spheres radius of GES = {} kpc with n = {}'.format(ss_radius2*f, 1000))

radius = r(positions, MW_centre)
idx, = np.where(radius < MW_rad)

MW_ids = ids[idx,]
MW_positions = positions[idx,:]
MW_masses = masses[idx,]

print('Fraction of stellar mass contained within GES at this point = {}'. format(np.sum(GES_masses)/GES_stellarmass[snapshots.index(snap)]))
print('Fraction of stellar mass contained within MW at this point = {}'. format(np.sum(MW_masses)/MW_stellarmass[snapshots.index(snap)]))

In [ ]:
GES_positions = A.util.CentreOnHalo(GES_positions, GES_centre)
fig, ax1 = plt.subplots(figsize=(4, 4))
n, xedges, yedges = np.histogram2d(GES_positions[:,2], GES_positions[:,1], bins=(500,500))
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
xc, yc = np.meshgrid(xbin, ybin)

ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')

plt.xlabel('x / kpc')
plt.ylabel('y / kpc')

In [ ]:
filenames = []
count = 0

b = 0.03
f = 10**3

for i in range(1280,1450,4):
    snap = i    

    positions,masses,ids = A.SnapshotData.load_snapshot(snap,4,['Coordinates', 'Masses', 'ParticleIDs']).get_results()
    MWcentre =  A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()
    
    if snap in list(GES_snaps):
        indice = list(GES_snaps).index(snap)
        GEScentre = GES_position[indice]
    else:
        GEScentre = MWcentre
        
    GEScentred = A.util.CentreOnHalo(GEScentre, MWcentre)
    time = A.util.time(snap)
            
    fig, ax1 = plt.subplots(figsize=(4,4))
    
    idx = np.where(np.isin(ids, GES_ids))[0]
    idx1 = np.where(np.isin(ids, GES_ids1))[0]
    idx2 = np.where(np.isin(ids, GES_ids2))[0]
    
    
    coords = positions[idx,:]
    mass0 = masses[idx,]
    coords = A.util.CentreOnHalo(coords, MWcentre)
    coords1 = positions[idx1,:]
    mass1 = masses[idx1,]
    coords1 = A.util.CentreOnHalo(coords1, MWcentre)
    coords2 = positions[idx2,:]
    mass2 = masses[idx2,]
    coords2 = A.util.CentreOnHalo(coords2, MWcentre)
    
    n, xedges, yedges = np.histogram2d(coords[:,0], coords[:,1], bins=(500,500),range=[[-b, b],[-b, b]] )
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma', alpha = 0.2)
    
    n, xedges, yedges = np.histogram2d(coords1[:,0], coords1[:,1], bins=(500,500),range=[[-b, b],[-b, b]] )
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='plasma')
    bary = A.ShrinkingSpheres.barycentre(coords1, mass1)


    n, xedges, yedges = np.histogram2d(coords2[:,0], coords2[:,1], bins=(500,500),range=[[-b, b],[-b, b]] )
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    ax1.contourf( xc*f, yc*f, np.log10(n.T), cmap='magma')
    bary = A.ShrinkingSpheres.barycentre(coords2, mass2)

    ax1.scatter(0,0, label = 'MW centre',s = 10, color = 'black')
    ax1.scatter(GEScentred[0]*f , GEScentred[1]*f, s= 6, color = 'blue', label = 'GES centre')
    ax1.scatter(bary[0]*f , bary[1]*f, s= 6, color = 'limegreen', label = 'Barycentre')
    
    ax1.set_xlim(-b*f,b*f)
    ax1.set_ylim(-b*f,b*f)
    
    ax1.set_xlabel('x / kpc')
    ax1.set_ylabel('y / kpc')
    
    ax1.legend(loc = 'upper right')
    
    plt.suptitle('Time = {} Gyr'.format(np.round(time,2)), fontsize =14)
    plt.title('n = 1000', fontsize = 12)

    count+=1
    
    name = 'Images_for_gif/{}.png'.format(count)
    plt.savefig(name)
    filenames.append(name)

    plt.close()